# Holt-Winters Exponential Smoothing

### Pre-Processing

In [3]:
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.dates import MonthLocator, DateFormatter

#Define the base directory and load datasets
notebook_dir = os.getcwd()
base_dir = os.path.join(notebook_dir, 'Hourly Energy Consumption')

#List of file names
file_names = [
    "AEP_hourly.csv", "COMED_hourly.csv", "DAYTON_hourly.csv",
    "DEOK_hourly.csv", "DOM_hourly.csv", "DUK_hourly.csv",
    "EKPC_hourly.csv", "FE_hourly.csv", "NI_hourly.csv",
    "PJM_Load_hourly.csv", "PJME_hourly.csv", "PJMW_hourly.csv"
]

#Load datasets into a dictionary
dataframes = {}
for file_name in file_names:
    file_path = os.path.join(base_dir, file_name)
    if os.path.exists(file_path):
        dataframes[file_name] = pd.read_csv(file_path)
        print(f"Loaded {file_name}")
    else:
        print(f"File not found: {file_name}")

#Combine all datasets into one DataFrame
combined_df = pd.concat(dataframes.values(), ignore_index=True)

#Preprocessing
if 'Datetime' in combined_df.columns:
    combined_df['Datetime'] = pd.to_datetime(combined_df['Datetime'])
else:
    raise ValueError("No 'Datetime' column found. Please check the dataset structure.")

#Fill in missing values
combined_df = combined_df.fillna(method='ffill')

File not found: AEP_hourly.csv
File not found: COMED_hourly.csv
File not found: DAYTON_hourly.csv
File not found: DEOK_hourly.csv
File not found: DOM_hourly.csv
File not found: DUK_hourly.csv
File not found: EKPC_hourly.csv
File not found: FE_hourly.csv
File not found: NI_hourly.csv
File not found: PJM_Load_hourly.csv
File not found: PJME_hourly.csv
File not found: PJMW_hourly.csv


ValueError: No objects to concatenate